<a href="https://colab.research.google.com/github/Rachita-G/Python_Practice/blob/main/Packages/PuLP_Linear_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 5.4 MB/s 


In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/ZhijingEu/Optimizing_Capital_Budgeting_With_ILP_Methods/master/Scenario3_InputData.csv"
data = pd.read_csv(url)

In [ ]:
data

,ID,Programme,Type,Location,Region,CAPEX,NPV,CAPEX Yr1,CAPEX Yr2,CAPEX Yr3
0,1,Piping,AssetIntegrity,Facility J,Zone4,5,6.00,4,1,0
1,2,Instrumentation,AssetIntegrity,Facility F,Zone1,10,11.00,1,5,4
2,3,Mechanical,AssetIntegrity,Facility C,Zone3,10,14.00,2,4,4
3,4,Debottlenecking,Growth,Facility D,Zone4,5,7.00,5,0,0
4,5,Piping,AssetIntegrity,Facility C,Zone3,10,12.00,4,4,2
5,6,Expansion,Growth,Facility C,Zone3,15,20.00,7,8,0
6,7,Revamp,Growth,Facility F,Zone1,15,16.00,7,7,1
7,8,Piping,AssetIntegrity,Facility D,Zone4,2,5.00,1,1,0
8,9,Revamp at Facility E Start Yr1,Growth,Facility E,Zone5,3,4.00,2,1,0
9,10,Revamp at Facility E Start Yr2,Growth,Facility E,Zone5,3,3.75,0,2,1


In [ ]:
import pulp

# Create A Model
phasing = pulp.LpProblem("Maximise", pulp.LpMaximize)

In [ ]:
phasing

Maximise:
MAXIMIZE
None
VARIABLES

In [ ]:
Selection = pulp.LpVariable.dicts("Selection", data.index, cat='Binary') # selection which needs to change with specified type
Selection

{0: Selection_0,
 1: Selection_1,
 2: Selection_2,
 3: Selection_3,
 4: Selection_4,
 5: Selection_5,
 6: Selection_6,
 7: Selection_7,
 8: Selection_8,
 9: Selection_9,
 10: Selection_10}

In [ ]:
phasing

Maximise:
MAXIMIZE
None
VARIABLES

In [ ]:
phasing += pulp.lpSum(Selection[idx]*data.loc[idx]["NPV"] for idx in data.index)
phasing # adds objective function and constraint on data type of variables

Maximise:
MAXIMIZE
6.0*Selection_0 + 11.0*Selection_1 + 9.0*Selection_10 + 14.0*Selection_2 + 7.0*Selection_3 + 12.0*Selection_4 + 20.0*Selection_5 + 16.0*Selection_6 + 5.0*Selection_7 + 4.0*Selection_8 + 3.75*Selection_9 + 0.0
VARIABLES
0 <= Selection_0 <= 1 Integer
0 <= Selection_1 <= 1 Integer
0 <= Selection_10 <= 1 Integer
0 <= Selection_2 <= 1 Integer
0 <= Selection_3 <= 1 Integer
0 <= Selection_4 <= 1 Integer
0 <= Selection_5 <= 1 Integer
0 <= Selection_6 <= 1 Integer
0 <= Selection_7 <= 1 Integer
0 <= Selection_8 <= 1 Integer
0 <= Selection_9 <= 1 Integer

In [ ]:
# Set The Constraints
yearly_constraints = [20, 25, 15] 
phasing += sum([Selection[idx]*data.loc[idx]["CAPEX Yr1"] for idx in data.index]) <= yearly_constraints[0]
phasing += sum([Selection[idx]*data.loc[idx]["CAPEX Yr2"] for idx in data.index]) <= yearly_constraints[1]
phasing += sum([Selection[idx]*data.loc[idx]["CAPEX Yr3"] for idx in data.index]) <= yearly_constraints[2]
phasing

Maximise:
MAXIMIZE
6.0*Selection_0 + 11.0*Selection_1 + 9.0*Selection_10 + 14.0*Selection_2 + 7.0*Selection_3 + 12.0*Selection_4 + 20.0*Selection_5 + 16.0*Selection_6 + 5.0*Selection_7 + 4.0*Selection_8 + 3.75*Selection_9 + 0.0
SUBJECT TO
_C1: 4 Selection_0 + Selection_1 + 3 Selection_10 + 2 Selection_2
 + 5 Selection_3 + 4 Selection_4 + 7 Selection_5 + 7 Selection_6 + Selection_7
 + 2 Selection_8 <= 20

_C2: Selection_0 + 5 Selection_1 + 3 Selection_10 + 4 Selection_2
 + 4 Selection_4 + 8 Selection_5 + 7 Selection_6 + Selection_7 + Selection_8
 + 2 Selection_9 <= 25

_C3: 4 Selection_1 + Selection_10 + 4 Selection_2 + 2 Selection_4
 + Selection_6 + Selection_9 <= 15

VARIABLES
0 <= Selection_0 <= 1 Integer
0 <= Selection_1 <= 1 Integer
0 <= Selection_10 <= 1 Integer
0 <= Selection_2 <= 1 Integer
0 <= Selection_3 <= 1 Integer
0 <= Selection_4 <= 1 Integer
0 <= Selection_5 <= 1 Integer
0 <= Selection_6 <= 1 Integer
0 <= Selection_7 <= 1 Integer
0 <= Selection_8 <= 1 Integer
0 <= Selecti

In [ ]:
phasing += Selection[4] + Selection[10] == 1 # Either Project 5 or 10 must be chosen
phasing += Selection[3] == Selection[7] # Both Projects 4 and 8 must go together
phasing += Selection[8] + Selection[9] == 1 # Either Project 9a or 9b must be chosen
phasing += Selection[6] <= Selection[1] # Project 7 is optional choice as long as Project 2 is selected

In [ ]:
phasing

Maximise:
MAXIMIZE
6.0*Selection_0 + 11.0*Selection_1 + 9.0*Selection_10 + 14.0*Selection_2 + 7.0*Selection_3 + 12.0*Selection_4 + 20.0*Selection_5 + 16.0*Selection_6 + 5.0*Selection_7 + 4.0*Selection_8 + 3.75*Selection_9 + 0.0
SUBJECT TO
_C1: 4 Selection_0 + Selection_1 + 3 Selection_10 + 2 Selection_2
 + 5 Selection_3 + 4 Selection_4 + 7 Selection_5 + 7 Selection_6 + Selection_7
 + 2 Selection_8 <= 20

_C2: Selection_0 + 5 Selection_1 + 3 Selection_10 + 4 Selection_2
 + 4 Selection_4 + 8 Selection_5 + 7 Selection_6 + Selection_7 + Selection_8
 + 2 Selection_9 <= 25

_C3: 4 Selection_1 + Selection_10 + 4 Selection_2 + 2 Selection_4
 + Selection_6 + Selection_9 <= 15

_C4: Selection_10 + Selection_4 = 1

_C5: Selection_3 - Selection_7 = 0

_C6: Selection_8 + Selection_9 = 1

_C7: - Selection_1 + Selection_6 <= 0

VARIABLES
0 <= Selection_0 <= 1 Integer
0 <= Selection_1 <= 1 Integer
0 <= Selection_10 <= 1 Integer
0 <= Selection_2 <= 1 Integer
0 <= Selection_3 <= 1 Integer
0 <= Selection

In [ ]:
# Run The Solver(s)
%time phasing.solve() #equivalent to phasing.solve(pulp.PULP_CBC_CMD()) as CBC is PulP's default solver
pulp.LpStatus[phasing.status]

CPU times: user 1.57 ms, sys: 7.06 ms, total: 8.62 ms
Wall time: 28.1 ms


'Optimal'

In [ ]:
# Print our objective function value and Output Solution
print(pulp.value(phasing.objective))

72.75


In [ ]:
selection_yn = []
for idx in data.index:
  selection_yn.append(Selection[idx].value())
data['Selection Y/N'] = pd.Series(selection_yn)
data['Capex Yr1 Selected'] = data['Selection Y/N']*data['CAPEX Yr1']
data['Capex Yr2 Selected'] = data['Selection Y/N']*data['CAPEX Yr2']
data['Capex Yr3 Selected'] = data['Selection Y/N']*data['CAPEX Yr3']
data['NPV Selected'] = data['Selection Y/N']*data['NPV']

In [ ]:
data

,ID,Programme,Type,Location,Region,CAPEX,NPV,CAPEX Yr1,CAPEX Yr2,CAPEX Yr3,Selection Y/N,Capex Yr1 Selected,Capex Yr2 Selected,Capex Yr3 Selected,NPV Selected
0,1,Piping,AssetIntegrity,Facility J,Zone4,5,6.00,4,1,0,0.0,0.0,0.0,0.0,0.00
1,2,Instrumentation,AssetIntegrity,Facility F,Zone1,10,11.00,1,5,4,1.0,1.0,5.0,4.0,11.00
2,3,Mechanical,AssetIntegrity,Facility C,Zone3,10,14.00,2,4,4,1.0,2.0,4.0,4.0,14.00
3,4,Debottlenecking,Growth,Facility D,Zone4,5,7.00,5,0,0,1.0,5.0,0.0,0.0,7.00
4,5,Piping,AssetIntegrity,Facility C,Zone3,10,12.00,4,4,2,1.0,4.0,4.0,2.0,12.00
5,6,Expansion,Growth,Facility C,Zone3,15,20.00,7,8,0,1.0,7.0,8.0,0.0,20.00
6,7,Revamp,Growth,Facility F,Zone1,15,16.00,7,7,1,0.0,0.0,0.0,0.0,0.00
7,8,Piping,AssetIntegrity,Facility D,Zone4,2,5.00,1,1,0,1.0,1.0,1.0,0.0,5.00
8,9,Revamp at Facility E Start Yr1,Growth,Facility E,Zone5,3,4.00,2,1,0,0.0,0.0,0.0,0.0,0.00
9,10,Revamp at Facility E Start Yr2,Growth,Facility E,Zone5,3,3.75,0,2,1,1.0,0.0,2.0,1.0,3.75
